# U.S. Medical Insurance Costs

# Goals
The main goal of this project is to create a python script that will analyse a data set in a way to gain a meaningful insight into the data.

# Data
The data set being analysed is a set of data points relating to the health insurance of various users from around the USA.

As far as the developer's aware, this data was gathered ethically.

The data set contains 7 categories:
- age; Age of the user
- sex; Sex of the user
- bmi; Measure of body mass that can be used to gauge a variety of possible health conditions
- children; Number of children included in the users plan
- smoker; Has the user reported that they are a smoker?
- region; Regional location of the user
- charges; Annual cost of the insurance plan


# Analysis
Due to the nature of the data set, the obvious paths of inquiry would relate to the "charges" data point.
As this data set is related to the cost of various users health insurance, a clear analysis would be to investigate the relationship between each of the various factors of a user to their insurance cost. e.g.
- users location and it's relationship to cost
- age and it's relationship to cost
- bmi and it's relationship to cost

While a main factor can be analysed, the additional factors can't be ignored in this analysis.
The data points must be categorised and subcategorised as much as possible while still allowing for a meaningful comparison.

With this taken into account, the following data comparisons were chosen:
1. age to region
   1. Region age averages
      1. How they compare to each other
      2. How they compare to the nation average
   2. Average cost per age bracket per region
2. smoking to bmi
   1. percentage smokers per BMI bracket
      1. regional comparison
   2. comparison of costs per BMI range between smokers and non-smokers
      1. regional comparison
3. age to children
   1. per region analysis

In [2]:
import csv


# Age to Region variables
ageRegion = {}
aveAgeRegional = {}
aveAgeNational = {}
ageBracketRegional = {}
ageBracketNational = {}

# Smoking to BMI variables
smokerBMIRegional = {}
smokerBMINational = {}
nonSmokerBMIRegional = {}
nonSmokerBMINational = {}

# age to children
ageBracketChildrenRegional = {}
ageBracketChildrenNational = {}

In [ ]:
with open('insurance.csv', 'r') as csvFile:
    dataSet = csv.DictReader(csvFile)
    for row in dataSet:
        